In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from financial.momentum.models.kerasAdvanced import KerasAdvancedModelFactory
from financial.momentum.experiment.modelExperiment import ModelExperimentFactory
import os
from dotenv import load_dotenv
from financial.momentum.utilities import find_dotenv
from financial.momentum.utilities import metrics
import financial.data as fd
from financial.io.file.cache import FileCache
import pandas as pd
from financial.lab.experiment import Experiment
from sklearn.metrics import r2_score

import tensorflow as tf
from keras import backend as K
import gc

def reset_gpu():
    K.clear_session()
    tf.keras.backend.clear_session()
    gc.collect()

2025-07-07 19:46:13.076638: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-07 19:46:13.189788: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751910373.233069    8503 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751910373.245713    8503 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751910373.337682    8503 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
load_dotenv(dotenv_path=find_dotenv())

cache = os.environ["CACHE"]
model_path = os.environ["MODEL"]
print(model_path)
print(os.environ["DATA"])
datastore = fd.CachedDataStore(path=os.environ["DATA"], cache=FileCache(cache_path=cache+"/"))
print(datastore) 

/home/manidmt/Universidad/TFG/OTRI/models/keras
/home/manidmt/Universidad/TFG/OTRI/data
CachedDataStore with 946 data sources [cache stats: {'size': 0, 'hit': 0, 'miss': 0, 'write': 0, 'read': 0, 'update': 0}]


In [4]:
ticker = "^GSPC"
start_date = "1990-01-01"
end_date = "2023-12-31"

In [5]:
data = datastore.get_data(ticker, start_date, end_date)
target = data[90 + 20:]

In [6]:
factory = KerasAdvancedModelFactory()

In [7]:
from itertools import product

# Arquitecturas a probar
architectures = ["rnn", "lstm", "cnn", "cnn2d", "transformer"]

# Hiperparámetros a explorar
layer_options = [[32, 16], [64, 32], [64, 32, 16]]
activations = ["relu", "tanh"]
batch_sizes = [32, 64]
epochs = 20

def ticker_single_models(ticker):
    for arch, layers, activation, batch in product(architectures, layer_options, activations, batch_sizes):
        name = f"{arch}_layers{len(layers)}_act{activation}_bs{batch}_{ticker}_single"
        print(f"Entrenando {name}...")

        config = {
            "mode": "global",  
            "datastore": datastore,
            "ticker": ticker,
            "model_factory": factory,
            "name": name,
            "start_year": start_date,
            "end_year": end_date,
            "lookahead": 20,
            "horizon": 90,
            "model_params": {
                "architecture": arch,
                "topology": {
                    "layers": layers,
                    "activation": {
                        "hidden": activation,
                        "output": "linear"
                    }
                },
                "optimization": {
                    "optimizer": "adam",
                    "loss": "mean_squared_error",
                    "metrics": ["mae"],
                    "epochs": epochs,
                    "batch_size": batch,
                    "validation_split": 0.1
                }
            }
        }

        try:
            experiment = ModelExperimentFactory.create_experiment(config)
            experiment.run()
            preds = experiment.reconstruct_absolute_predictions_from_relative()
            metrics(experiment, preds, target)
            reset_gpu()
        except Exception as e:
            print(f"Falló el modelo {name}: {e}")

In [8]:
ticker_single_models("AAPL")

Entrenando rnn_layers2_actrelu_bs32_AAPL_single...


I0000 00:00:1751910377.234574    8503 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6257 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Architecture: rnn
X_train before fit: (5959, 90, 1), y_train: (5959, 1)
Number of 5959 samples, 90 timesteps, 1 features
Epoch 1/300


I0000 00:00:1751910378.609754   22908 service.cc:152] XLA service 0x7ac8e800bf90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751910378.609768   22908 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2025-07-07 19:46:18.649874: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1751910378.797108   22908 cuda_dnn.cc:529] Loaded cuDNN version 90300


 9/47 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.0067 - loss: 0.0842

I0000 00:00:1751910380.542296   22908 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


42/47 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0042 - loss: 0.0546

2025-07-07 19:46:22.355188: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_64', 104 bytes spill stores, 104 bytes spill loads

2025-07-07 19:46:22.466907: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_64', 632 bytes spill stores, 632 bytes spill loads



47/47 ━━━━━━━━━━━━━━━━━━━━ 5s 54ms/step - accuracy: 0.0041 - loss: 0.0527
Epoch 2/300
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.0053 - loss: 0.0251
Epoch 3/300
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.0024 - loss: 0.0267
Epoch 4/300
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.0034 - loss: 0.0257
Epoch 5/300
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.0039 - loss: 0.0245
Epoch 6/300
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.0037 - loss: 0.0279
Epoch 7/300
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.0037 - loss: 0.0268
Epoch 8/300
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.0026 - loss: 0.0261
Epoch 9/300
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.0056 - loss: 0.0320
Epoch 10/300
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.0032 - loss: 0.0228
Epoch 11/300
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.0034 - loss: 0.0302
Epoch 12/300
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy:

KeyboardInterrupt: 